In [15]:
class ForwardChainingFOL:
    def __init__(self):
        # Facts in the knowledge base
        self.facts = set()
        # Rules in the knowledge base
        self.rules = []

    def add_fact(self, fact):
        """
        Add a fact to the knowledge base.
        """
        self.facts.add(fact)

    def add_rule(self, premises, conclusion):
        """
        Add a rule to the knowledge base.
        premises: List of premises (conditions).
        conclusion: The conclusion of the rule.
        """
        self.rules.append((premises, conclusion))

    def unify(self, fact, premise):
        """
        Attempt to unify a fact with a rule's premise.
        """
        if "(" not in premise:  # If premise is a constant
            return fact == premise
        if premise.split("(")[0] != fact.split("(")[0]:  # Predicate mismatch
            return None

        # Extract arguments
        fact_args = fact[fact.index("(") + 1 : -1].split(",")
        premise_args = premise[premise.index("(") + 1 : -1].split(",")
        if len(fact_args) != len(premise_args):
            return None

        substitution = {}
        for f_arg, p_arg in zip(fact_args, premise_args):
            if p_arg.islower():  # Variable in premise
                if p_arg in substitution:
                    if substitution[p_arg] != f_arg:
                        return None
                else:
                    substitution[p_arg] = f_arg
            elif f_arg != p_arg:  # Constant mismatch
                return None
        return substitution

    def apply_substitution(self, expr, substitution):
        """
        Apply a substitution to an expression.
        """
        if "(" not in expr:
            return substitution.get(expr, expr)

        predicate = expr.split("(")[0]
        args = expr[expr.index("(") + 1 : -1].split(",")
        substituted_args = [substitution.get(arg, arg) for arg in args]
        return f"{predicate}({','.join(substituted_args)})"

    def infer(self):
        """
        Perform forward chaining to infer all possible facts.
        """
        new_inferences = True
        while new_inferences:
            new_inferences = False
            for premises, conclusion in self.rules:
                substitutions = []
                for premise in premises:
                    for fact in self.facts:
                        subst = self.unify(fact, premise)
                        if subst is not None:
                            substitutions.append(subst)
                            break

                if len(substitutions) == len(premises):
                    combined_subst = {}
                    for subst in substitutions:
                        combined_subst.update(subst)

                    inferred_fact = self.apply_substitution(conclusion, combined_subst)
                    if inferred_fact not in self.facts:
                        print(f"Inferred: {inferred_fact}")
                        self.facts.add(inferred_fact)
                        new_inferences = True

    def query(self, query_fact):
        """
        Query if a specific fact can be inferred.
        """
        if query_fact in self.facts:
            print(f"Proved: {query_fact}")
            return True
        else:
            print(f"Could not prove: {query_fact}")
            return False


# Create the knowledge base for the criminal example
kb = ForwardChainingFOL()

# Add facts
kb.add_fact("American(Robert)")
kb.add_fact("Missile(T1)")
kb.add_fact("Owns(A,T1)")
kb.add_fact("Enemy(A,America)")

# Add rules
kb.add_rule(["Missile(x)", "Owns(A,x)"], "Sells(Robert,x,A)")
kb.add_rule(["Missile(x)"], "Weapon(x)")
kb.add_rule(["Enemy(x,America)"], "Hostile(x)")
kb.add_rule(["American(p)", "Weapon(q)", "Sells(p,q,r)", "Hostile(r)"], "Criminal(p)")

# Perform inference
kb.infer()

# Query the system
kb.query("Criminal(Robert)")


Inferred: Sells(Robert,T1,A)
Inferred: Weapon(T1)
Inferred: Hostile(A)
Inferred: Criminal(Robert)
Proved: Criminal(Robert)


True